In [108]:
import pandas as pd
import re
from urllib.request import urlopen
import json
import requests

import spacy
from spacy.tokenizer import Tokenizer

from spacy.tokens import Span
from spacy.matcher import Matcher


In [53]:
df = pd.read_csv('cleaned.csv')
# skills = pd.read_csv('skills.csv')

In [54]:
url = 'http://127.0.0.1:5277/role/get_all'

res = urlopen(url).read()
content = json.loads(res)['content']
roles = []

for text in content: 
    roles.append(text['name'])

In [55]:
rows_to_keep = []

for role in roles:  
    found = df['jobtitle'].notna() & df.jobtitle.str.contains(role, case=False)
    if role == "UI/UX Desiigner":
        found = df['jobtitle'].notna() & (df.jobtitle.str.contains("UI Designer", case=False) | df.jobtitle.str.contains("UX Designer", case=False))
    elif role == "Pre-Sales Consultant": 
        found = df['jobtitle'].notna() & df.jobtitle.str.contains("presale", case=False)
        
    df.loc[found, 'jobtitle'] = role
    
    temp = df[df.jobtitle.str.contains(role, case=False) == True].index 
    print(role, len(temp))
    for idx in temp: 
        if idx not in rows_to_keep: 
            rows_to_keep.append(idx)

df = df.loc[rows_to_keep]
df.reset_index(drop=True, inplace=True)
df.drop(columns=['company'], inplace=True)

Business Analyst 583
Data Analyst 161
Data Engineer 98
Pre-Sales Consultant 3
Security Engineer 216
Software Engineer 842
UI/UX Desiigner 119


In [56]:
df

,jobtitle,jobdescription,skills
0,Business Analyst,Genesis10 is looking for a Business Analyst fo...,"Analysis, Analyst, Application, Business Analy..."
1,Business Analyst,Genesis10's G10 Associates Program is seeking ...,"Analyst, Business Analyst, Business Requiremen..."
2,Business Analyst,PLEASE JOIN OUR TALENT NETWORK: http://bit.ly/...,SQL Reporting Services Suite
3,Business Analyst,VanderHouwen has more jobs you may like! Find ...,Telecommuting not available Travel not required
4,Business Analyst,Job Description: Seeking a Business Process An...,"Mortgage/Equiting Lending, Lean Six Sigma Gree..."
...,...,...,...
2017,UI/UX Desiigner,Position: UX Designer (Agency / Axure skills n...,"Axure, Photoshop, InDesign & Illustrator"
2018,UI/UX Desiigner,LOOKING FOR A UX DESIGNER TO WORK IN THE CUSTO...,UX FRAMEWORKS FRONT-END DEV SKETCH VISION
2019,UI/UX Desiigner,My Direct Client is looking for a UX Designer ...,"web design, wireframe, wireframming, design, r..."
2020,UI/UX Desiigner,The ideal candidate will have experience desig...,"UI UX ""user experience"" gui ""user interface"" g..."


In [57]:
biz_analyst = df[df['jobtitle'] == 'Business Analyst']['jobdescription'].reset_index(drop=True)
data_analyst = df[df['jobtitle'] == 'Data Analyst']['jobdescription'].reset_index(drop=True)
data_engineer = df[df['jobtitle'] == 'Data Engineer']['jobdescription'].reset_index(drop=True)
pre_sales_consultant = df[df['jobtitle'] == 'Pre-Sales Consultant']['jobdescription'].reset_index(drop=True)
security_engineer = df[df['jobtitle'] == 'Security Engineer']['jobdescription'].reset_index(drop=True)
swe = df[df['jobtitle'] == 'Software Engineer']['jobdescription'].reset_index(drop=True)
ui_ux = df[df['jobtitle'] == 'UI/UX Desiigner']['jobdescription'].reset_index(drop=True)

In [90]:
spacy_model = spacy.load("en_core_web_lg")
matcher = Matcher(spacy_model.vocab)

extraction_terms = ['experience', 'familiar', 'knowledge', 'proficient', 'understand', 'skilled', 'ability', 'expertise', 'proficiency', 'competency']

patterns = [
    [{'POS': 'NOUN', 'OP': '+'}, {'LOWER': {'IN': extraction_terms}, 'OP': '+'}, {'POS': {'IN': ['NOUN', 'PROPN', 'ADJ']}, 'OP': '+'}],
    [{'LOWER': {'IN': extraction_terms}, 'OP': '+'}, {'POS': 'ADP'}, {'POS': {'IN': ['DET', 'NOUN', 'PROPN']}, 'OP': '+'}],
    [{'LOWER': 'or'}, {'POS': 'NOUN', 'OP': '+'}, {'POS': {'IN': ['NOUN', 'PROPN', 'ADJ']}, 'OP': '+'}],
    [{'LOWER': 'or'}, {'LOWER': {'IN': extraction_terms}, 'OP': '+'}, {'POS': 'ADP'}, {'POS': {'IN': ['DET', 'NOUN', 'PROPN']}, 'OP': '+'}]
]

for i, pattern in enumerate(patterns):
    matcher.add(f'skill_pattern_{i}', [pattern])
    
word_to_exclude = ['business', 'requirements', 'applications', 'years', 'use', 'analysis',
                  'processes','Business', 'design','systems issues', 'implementation',
                   'phases','systems','procedures','technology','disability status','tools',
                   'understanding','field','technologies','techniques','variety','visa assistance',
                   'Medicare','development','end','areas','specific experience','object',
                   'accessibility','nuances'
                  ]

In [191]:
def get_left_span(token, label='', include=True):
    offset = 1 if include else 0
    idx = token.i
    while idx > token.left_edge.i:
        if token.doc[idx - 1].pos_ in ('NOUN', 'PROPN', 'ADJ', 'X'):
            idx -= 1
        else:
            break
    return label, idx, token.i + offset

# get all the related words that are connected with the conjunctions
def get_conjugations(token):
    new = [token]
    while new:
        token = new.pop()
        yield token
        for child in token.children:
            if child.dep_ == 'conj':
                new.append(child)

# extracts all terms related to extraction_terms
def extract_adp_conj_experience(doc):
    for token in doc:
        if token.lower_ in extraction_terms:
            for child in token.rights:
                if child.dep_ == 'prep':
                    for obj in child.children:
                        if obj.dep_ == 'pobj':
                            for conj in get_conjugations(obj):
                                yield get_left_span(conj, token.lower_)
                                
def to_camel_case(s):
    words = s.split()
    camel_case = ' '.join(word.capitalize() if word.islower() else word for word in words)
    return camel_case

def get_skills(li, role): 
    dic = {}

    for i in range(len(li)): 
        # Apply the matcher to the text
        doc = spacy_model(li[i])

        # Find all noun, pronoun and adj
        matches = matcher(doc)

        # Find all conjunction skills
        additional_skills = list(extract_adp_conj_experience(doc))

        matched_terms = []
        prev_phrase = ''
        current_phrase = ''

        # Extract skills from patterns
        for match_id, start, end in matches:
            phrase_tokens = doc[start:end]
            current_phrase = str(phrase_tokens)        

            # If "or" is present, consider only the tokens after "or"
            if "or" in [token.text.lower() for token in phrase_tokens]:
                or_index = [token.i for token in phrase_tokens if token.text.lower() == 'or'][0]
                filtered_tokens = [token.text for token in doc[or_index + 1:end] if
                                   token.pos_ in ['NOUN', 'PROPN'] and token.text.lower() not in extraction_terms]
            else:
                filtered_tokens = [token.text for token in phrase_tokens if
                                   token.pos_ in ['NOUN', 'PROPN'] and token.text.lower() not in extraction_terms]

            if filtered_tokens: 
                most_specific_term = ' '.join(filtered_tokens)

                #checks if the prev_phrase is a sub substring of the current_phrase
                if prev_phrase in current_phrase and len(matched_terms) != 0:
                    matched_terms.pop()
                    
                matched_terms.append(to_camel_case(most_specific_term))

            prev_phrase = current_phrase

        prev_phrase = ''

        for _, start, end in additional_skills: 
            phrase_tokens = doc[start:end]
            current_phrase = str(phrase_tokens)

            if current_phrase not in extraction_terms and str(phrase_tokens) not in matched_terms and (prev_phrase not in current_phrase or prev_phrase == ''): 
                matched_terms.append(to_camel_case(str(phrase_tokens)))
                
            elif current_phrase not in extraction_terms and str(phrase_tokens) not in matched_terms and prev_phrase in current_phrase: 
                matched_terms.pop()
                matched_terms.append(to_camel_case(str(phrase_tokens)))

            prev_phrase = current_phrase

        for phrases in matched_terms: 
            if phrases in dic: 
                dic[phrases] += 1
            else: 
                dic[phrases] = 1
                
    word_to_exclude.append(role)

    sorted_dic = dict(sorted(dic.items(), key=lambda item: item[1], reverse=True))
    filtered_dic = {key: value for key, value in sorted_dic.items() if all(word.lower() not in key.lower() for word in word_to_exclude)}
    return dict(list(filtered_dic.items())[:25])

In [ ]:
li = [biz_analyst, data_analyst, data_engineer, pre_sales_consultant, security_engineer, swe, ui_ux]
dic = {}

for i in range(len(df.jobtitle.unique())): 
    title = df.jobtitle.unique()[i]
    dic[title] = get_skills(li[i], title)

In [ ]:
json_string = json.dumps(dic, indent=4)
file_path = 'output.json'

with open(file_path, 'w') as file:
    file.write(json_string)

print(f'Dictionary saved to {file_path}')

In [105]:
backend = 'http://127.0.0.1:5277/'

Create Keyword

In [140]:
skills = []
idx = 0
create_url = backend + "keyword/create"

# for role in dic.keys(): 
#     for skill in dic[role].keys(): 
#         if skill not in skills:
#             skills.append(skill)
#             data = {'keyword_id': str(idx), 'keyword_name': skill}
#             idx += 1
#             response = requests.post(create_url, json=data)
#             if response.status_code == 200:
#                 pass
#             else:
#                 print(f'POST request failed with status code {response.status_code} {skill}')

for role in dic.keys(): 
    for skill in dic[role].keys(): 
        if skill not in skills:
            skills.append(skill)
            data = {'keyword_id': str(idx), 'keyword_name': skill}
            idx += 1
            response = requests.post(create_url, json=data)
            if response.status_code == 200:
                pass
            else:
                print(f'POST request failed with status code {response.status_code} {skill}')

POST request failed with status code 401 SQL
POST request failed with status code 401 Excel
POST request failed with status code 401 project management
POST request failed with status code 401 Agile
POST request failed with status code 401 PowerPoint
POST request failed with status code 401 Oracle
POST request failed with status code 401 testing
POST request failed with status code 401 Salesforce.com
POST request failed with status code 401 MS Project
POST request failed with status code 401 data
POST request failed with status code 401 MS Word
POST request failed with status code 401 IT
POST request failed with status code 401 Waterfall
POST request failed with status code 401 Telecom
POST request failed with status code 401 MS Excel
POST request failed with status code 401 MS Visio
POST request failed with status code 401 writing
POST request failed with status code 401 Capital Markets
POST request failed with status code 401 Agile methodologies
POST request failed with status code 4

Add keyword to role

In [169]:
add_to_role_url = backend + "role/keyws_mapped/create"

skillRes = requests.get(backend + "keyword/get_all").text
skillData = json.loads(skillRes)["content"]

roleRes = requests.get(backend + "role/get_all").text
roleData = json.loads(roleRes)["content"]

def find_skill_id(search): 
    for skill in skillData:
        if skill["name"].lower() == search.lower():
            return skill["id"]
        
def find_role_id(search): 
    for role in roleData:
        if role["name"].lower() == search.lower():
            return role["id"]
        
delete_keyword_role_url = backend + "role/keyws_mapped/delete"

for role in dic.keys(): 
    role_id = find_role_id(role)
    roleKeyRes = requests.get(backend + "role/keyws_mapped/get/" + str(role_id)).text
    roleKeyData = json.loads(roleKeyRes)["content"]
    li = []
    for keyword in roleKeyData: 
        li.append(keyword['id'])
    data = {"role_id": str(role_id), "keyw_id": li}
        
    response = requests.post(delete_keyword_role_url, json=data)
    if response.status_code == 200:
        pass
    else:
        print(f'POST request failed with status code {response.status_code} {li}')
        
for role in dic.keys(): 
    role_id = find_role_id(role)
    keyword_id = []
    for skill in list(dic[role].keys())[:3]:
        print(skill)
        keyword_id.append(find_skill_id(skill))
    data = {"role_id": str(role_id), "keyw_id": keyword_id}
    response = requests.post(add_to_role_url, json=data)
    if response.status_code == 200:
        pass
    else:
        print(f'POST request failed with status code {response.status_code} {skill}')

SQL
Excel
project management
POST request failed with status code 201 project management
SQL
data
Excel
POST request failed with status code 201 Excel
Spark
ETL
Kafka
POST request failed with status code 201 Kafka
SAP
Opportunity Management
Analytics solutions
POST request failed with status code 201 Analytics solutions
Linux
Information Security
security
POST request failed with status code 201 security
Java
JavaScript
Linux
POST request failed with status code 201 Linux
HTML
Axure
Sketch
POST request failed with status code 201 Sketch


Camel Case

In [189]:
get_keyword_url = backend + "keyword/get_all"
getRes = requests.get(get_keyword_url).text
getData = json.loads(getRes)["content"]

update_url = backend + "keyword/update"

for keyword in getData: 
    camel_case = to_camel_case(keyword['name'])
    data = {'keyword_id': keyword['id'], 'keyword_name': camel_case}
    response = requests.post(update_url, json=data)
    if response.status_code == 200:
        pass
    else:
        print(f'POST request failed with status code {response.status_code} {data}')


POST request failed with status code 201 {'keyword_id': 106, 'keyword_name': '.NET'}
POST request failed with status code 201 {'keyword_id': 41, 'keyword_name': 'Abilities'}
POST request failed with status code 201 {'keyword_id': 116, 'keyword_name': 'Adobe Creative Suite'}
POST request failed with status code 201 {'keyword_id': 63, 'keyword_name': 'Advanced SQL Programming'}
POST request failed with status code 201 {'keyword_id': 7, 'keyword_name': 'Agile'}
POST request failed with status code 201 {'keyword_id': 18, 'keyword_name': 'Agile Methodologies'}
POST request failed with status code 201 {'keyword_id': 67, 'keyword_name': 'Analytics Solutions'}
POST request failed with status code 201 {'keyword_id': 109, 'keyword_name': 'AngularJS'}
POST request failed with status code 201 {'keyword_id': 90, 'keyword_name': 'Application'}
POST request failed with status code 201 {'keyword_id': 83, 'keyword_name': 'Application Security'}
POST request failed with status code 201 {'keyword_id': 9,

POST request failed with status code 201 {'keyword_id': 135, 'keyword_name': 'Prototypes'}
POST request failed with status code 201 {'keyword_id': 43, 'keyword_name': 'Python'}
POST request failed with status code 201 {'keyword_id': 133, 'keyword_name': 'Rapid Prototyping'}
POST request failed with status code 201 {'keyword_id': 36, 'keyword_name': 'Recruiters'}
POST request failed with status code 201 {'keyword_id': 50, 'keyword_name': 'Relational Databases'}
POST request failed with status code 201 {'keyword_id': 19, 'keyword_name': 'Reporting'}
POST request failed with status code 201 {'keyword_id': 131, 'keyword_name': 'Research'}
POST request failed with status code 201 {'keyword_id': 70, 'keyword_name': 'Sales Engineering'}
POST request failed with status code 201 {'keyword_id': 65, 'keyword_name': 'SAP'}
POST request failed with status code 201 {'keyword_id': 26, 'keyword_name': 'SAS'}
POST request failed with status code 201 {'keyword_id': 64, 'keyword_name': 'Scala'}
POST requ

[{'id': 106, 'name': '.NET'},
 {'id': 41, 'name': 'Abilities'},
 {'id': 116, 'name': 'Adobe Creative Suite'},
 {'id': 63, 'name': 'advanced SQL programming'},
 {'id': 7, 'name': 'Agile'},
 {'id': 18, 'name': 'Agile methodologies'},
 {'id': 67, 'name': 'Analytics solutions'},
 {'id': 109, 'name': 'AngularJS'},
 {'id': 90, 'name': 'application'},
 {'id': 83, 'name': 'application security'},
 {'id': 9, 'name': 'Automation'},
 {'id': 113, 'name': 'AWS'},
 {'id': 114, 'name': 'Axure'},
 {'id': 33, 'name': 'BI'},
 {'id': 126, 'name': 'Bootstrap'},
 {'id': 132, 'name': 'branding'},
 {'id': 125, 'name': 'broad range'},
 {'id': 100, 'name': 'C'},
 {'id': 103, 'name': 'C #'},
 {'id': 111, 'name': 'C#'},
 {'id': 104, 'name': 'C++'},
 {'id': 17, 'name': 'Capital Markets'},
 {'id': 71, 'name': 'Carrier Ethernet'},
 {'id': 38, 'name': 'coding'},
 {'id': 37, 'name': 'concepts'},
 {'id': 4, 'name': 'Cryptography'},
 {'id': 102, 'name': 'CSS'},
 {'id': 222, 'name': 'data'},
 {'id': 40, 'name': 'data an

In [151]:
skillData

[{'id': 106, 'name': '.NET'},
 {'id': 41, 'name': 'Abilities'},
 {'id': 116, 'name': 'Adobe Creative Suite'},
 {'id': 63, 'name': 'advanced SQL programming'},
 {'id': 7, 'name': 'Agile'},
 {'id': 18, 'name': 'Agile methodologies'},
 {'id': 67, 'name': 'Analytics solutions'},
 {'id': 109, 'name': 'AngularJS'},
 {'id': 90, 'name': 'application'},
 {'id': 83, 'name': 'application security'},
 {'id': 9, 'name': 'Automation'},
 {'id': 113, 'name': 'AWS'},
 {'id': 114, 'name': 'Axure'},
 {'id': 33, 'name': 'BI'},
 {'id': 126, 'name': 'Bootstrap'},
 {'id': 132, 'name': 'branding'},
 {'id': 125, 'name': 'broad range'},
 {'id': 100, 'name': 'C'},
 {'id': 103, 'name': 'C #'},
 {'id': 111, 'name': 'C#'},
 {'id': 104, 'name': 'C++'},
 {'id': 17, 'name': 'Capital Markets'},
 {'id': 71, 'name': 'Carrier Ethernet'},
 {'id': 38, 'name': 'coding'},
 {'id': 37, 'name': 'concepts'},
 {'id': 4, 'name': 'Cryptography'},
 {'id': 102, 'name': 'CSS'},
 {'id': 40, 'name': 'data analytics'},
 {'id': 21, 'name':

In [104]:
skills

['SQL',
 'Excel',
 'project management',
 'Agile',
 'PowerPoint',
 'Oracle',
 'testing',
 'Salesforce.com',
 'MS Project',
 'data',
 'MS Word',
 'IT',
 'Waterfall',
 'Telecom',
 'MS Excel',
 'MS Visio',
 'writing',
 'Capital Markets',
 'Agile methodologies',
 'reporting',
 'HCLS Payer',
 'data mapping',
 'system',
 'operations',
 'ETL',
 'scripting',
 'SAS',
 'data modeling',
 'data mining',
 'databases',
 'enterprise reporting',
 'fine tune database performance',
 'SQL Server',
 'BI',
 'Informatica',
 'IT staffing',
 'Recruiters',
 'concepts',
 'coding',
 'practices',
 'data analytics',
 'Abilities',
 'SQLin order',
 'python',
 'programming',
 'Spark',
 'Kafka',
 'scripting languages',
 'Python',
 'Hadoop',
 'relational databases',
 'data warehousing concepts',
 'Data Warehousing',
 'Hadoop ecosystem',
 'Tableau',
 'statistics',
 'data structures',
 'Microsoft Office',
 'Java',
 'Data Modeling',
 'data transformation',
 'Hive',
 'Hue',
 'advanced SQL programming',
 'Scala',
 'SAP',
 '

{'business analysis': 18, 'SQL': 17, 'business': 16, 'requirements': 14, 'Excel': 13, 'applications': 12, 'years': 12, 'use': 12, 'analysis': 11, 'project management': 9}

{'SQL': 14, 'data': 8, 'data analysis': 7, 'tools': 5, 'analysis': 5, 'processes': 5, 'Excel': 4, 'procedures': 4, 'scripting': 4, 'technologies': 4}

{'Spark': 7, 'design': 7, 'development': 6, 'ETL': 6, 'Kafka': 6, 'scripting languages': 5, 'Python': 5, 'Hadoop': 5, 'relational databases': 5, 'SQL': 4}

{'SAP Product suite areas': 2, 'SAP': 2, 'Opportunity Management': 2, 'end': 2, 'Analytics solutions': 2, 'implementation experience': 2, 'various SAP modules': 2, 'solutions': 1, 'sales engineering': 1, 'Carrier Ethernet': 1}

{'applications': 14, 'security systems': 13, 'tools': 12, 'areas': 10, 'Linux': 10, 'Information Security': 8, 'design': 7, 'operating systems': 7, 'security': 7, 'TCP': 7}

{'Java': 75, 'development': 57, 'design': 39, 'JavaScript': 38, 'Linux': 36, 'C': 34, 'HTML': 29, 'SQL': 28, 'object': 

In [63]:
# new_dic = {}

# for role in dic.keys(): 
#     temp = {}
#     for skill in dic[role].keys(): 
#         if len(skills[skills.preferredLabel.str.lower() == skill.lower()]) or len(skills[skills.altLabels.str.lower() == skill.lower()]): 
#             temp[skill] = dic[role][skill]
#     new_dic[role] = temp
    

In [64]:
# new_dic

{'Business Analyst': {'SQL': 17, 'database': 3, 'SQL Server': 3},
 'Data Analyst': {'SQL': 14,
  'data mining': 3,
  'SQL Server': 3,
  'data analytics': 2,
  'business intelligence': 2},
 'Data Engineer': {'Spark': 7,
  'Hadoop': 5,
  'SQL': 4,
  'Scala': 3,
  'machine learning': 2,
  'Business Intelligence': 2,
  'Computer Science': 2},
 'Pre-Sales Consultant': {},
 'Security Engineer': {'operating systems': 7},
 'Software Engineer': {'JavaScript': 38,
  'SQL': 28,
  'CSS': 21,
  'C++': 21,
  'Hadoop': 17,
  'MySQL': 17,
  'SQL Server': 13,
  'PHP': 12,
  'C#': 12,
  'Javascript': 8,
  'machine learning': 7,
  'Scala': 7,
  'database': 6},
 'UI/UX Desiigner': {'JavaScript': 5,
  'database': 4,
  'CSS': 2,
  'Adobe Photoshop': 2}}